In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multivariate_normal
import math
from numpy.linalg import inv
from numpy.random import multivariate_normal as npmul

In [3]:
def prediction(x, F, sqrtU):
    return np.matmul(F, x) + np.matmul(sqrtU, np.random.randn(sqrtU.shape[1]))

def observation(x, H, sqrtV):
    return np.matmul(H, x) + np.matmul(sqrtV, np.random.randn(sqrtV.shape[1]))

def Kalman_filter(y, m0, P0, F, H, U, V, n, xdim, delta):
    # Updated mean and variance
    mu = np.zeros((xdim, n))
    Pu = np.zeros((xdim, xdim, n))
    # Predicted mean and variance
    mp = np.zeros((xdim, n))
    Pp = np.zeros((xdim, xdim, n))
    mp[:, 0] = m0
    Pp[:, :, 0] = P0
    for k in range(n):
        if k < n - delta:
            # Update
            z = y[:, k] - np.matmul(H, mp[:, k])
            S = np.matmul(np.matmul(H, Pp[:, :, k]), H.transpose()) + V
            K = np.matmul(np.matmul(Pp[:, :, k], H.transpose()), inv(S))
            mu[:, k] = mp[:, k] + np.matmul(K, z)
            Pu[:, :, k] = np.matmul(np.eye(xdim) - np.matmul(K, H), Pp[:, :, k])
        else:
            mu[:, k] = mp[:, k]
            Pu[:, :, k] = Pp[:, :, k]
        
        if k < n-1:
            # Prediction
            mp[:, k+1] = np.matmul(F, mu[:, k])
            Pp[:, :, k+1] = np.matmul(np.matmul(F, Pu[:, :, k]), F.transpose()) + U
    return mp, Pp, mu, Pu

def generate_data(F, H, sqrtU, sqrtV, n, xdim, ydim):
    x = np.zeros((xdim, n))
    y = np.zeros((ydim, n))
    x[:, 0] = x0
    y[:, 0] = observation(x0, H, sqrtV)
    for k in range(1,n):
        x[:, k] = prediction(x[:, k-1], F, sqrtU)
        y[:, k] = observation(x[:, k], H, sqrtV)
    return x, y

In [4]:
def multiplication(a,b):
    A = []
    for i in range(b.shape[0]):
        s = np.matmul(a,b[i])
        A.append(s)
    return A

def pdfmul(x,mean,var):
    A = []
    for i in range(len(mean)):
        s = multivariate_normal(mean[i], var).pdf(x)
        A.append(s)
    return A

def genmul(mean,var,N):
    A=[]
    for i in range(N):
        a = npmul(mean[i],var,1)[0]
        A.append(a)
    return np.array(A)

def resam(x,weight,N):
    w = np.random.choice(np.arange(0,N), N, p= weight) #to find which particle to resampling
    for k in np.arange(0,N):
        x[k] = x[w[k]]
    return x

In [5]:
# Dimensions of the state and observation
xdim = 2
ydim = 1

# Initial state
x0 = np.array([0.0, 0.5])

# Observation matrix
H = np.array([[1.0, 0.0]])

# Observation noise
sigma = 1.0
sqrtV = np.array([[sigma]])
V = np.matmul(sqrtV, sqrtV.transpose())

# Prior
m0 = np.array([0.0, 0.0])
P0 = np.array([[1.0, 0.0], [0.0, 0.25]])

# Evolution
dt = 1.0

acc_noise = 0.05
F = np.array([[1.0, dt], [0.0, 1.0]])
sqrtU = acc_noise * np.array([[dt**2/2],[dt]])
U = np.matmul(sqrtU, sqrtU.transpose()) 

# Time steps
n = 100
delta = 0

# Data generation
(x, y) = generate_data(F, H, sqrtU, sqrtV, n, xdim, ydim)

# Kalman filter
(mp, Pp, mu, Pu) = Kalman_filter(y, m0, P0, F, H, U, V, n, xdim, delta)

#new construction of x ([x1,x2])
Xsys=[]
for j in range(n):
    Xsys.append([x[0][j],x[1][j]])
Ysys=y[0]

In [6]:
def SIS(T, N): # T = timestep  N = number of particles
    Xsis = np.zeros((T,N,xdim)) #matrix of particles for 100 times
    Wsis = np.zeros((T,N)) #matrix of weight of particles for 100 times
    Meansis1 = np.zeros(T) 
    Meansis2 = np.zeros(T) 
    
    xsis0 = npmul(m0,P0,N) #q(x1|y1)= u(x1) sample particle for first time
    alsis0 = pdfmul(Ysys[0],multiplication(H,xsis0),V)
    wsis0 = alsis0/np.sum(alsis0)  #w1=g(y1|x1)
    Xsis[0] = xsis0
    Wsis[0] = wsis0
    
    for t in range(0,T-1): 
        xsis = genmul(multiplication(F,Xsis[t]),U,N) #q(xn|yn,xn-1) = f(xn|xn-1)
        alsis = pdfmul(Ysys[t+1],multiplication(H,xsis),V) #calculate incremental weight \alpha_n = g(yn|xn)
        wsis = alsis*Wsis[t]/np.sum(alsis*Wsis[t]) #weight of each particl at time n #wn=\alpha_n * wn-1
        Xsis[t+1] = xsis
        Wsis[t+1] = wsis
        
    for k in range(0,T):
        Meansis1[k] = np.sum(Xsis[k][:,0]*Wsis[k]) #filtering meanx1 time k
        Meansis2[k] = np.sum(Xsis[k][:,1]*Wsis[k]) #filtering meanx2 time k
    return Xsis, Wsis, Meansis1, Meansis2

def SMC(T, N):
    Xsmc = np.zeros((T,N,xdim)) #matrix of particles for 100 times
    Wsmc = np.zeros((T,N)) #matrix of weight of particles for 100 times
    Meansmc1 = np.zeros(T) 
    Meansmc2 = np.zeros(T) 
    Xb = np.zeros((T,N,xdim)) #matrix of resampling particles

    xsmc0 = npmul(m0,P0,N) #q(x1|y1)= u(x1) sample particle for first time
    alsmc0 = pdfmul(Ysys[0],multiplication(H,xsmc0),V)
    wsmc0 = alsmc0/np.sum(alsmc0)  #w1=g(y1|x1)
    Xsmc[0] = xsmc0
    Wsmc[0] = wsmc0
    Xb[0] = resam(Xsmc[0], Wsmc[0], N)

    for t in range(0,T-1): 
        xsmc = genmul(multiplication(F,Xb[t]),U,N)  #q(xn|yn,xn-1) = f(xn|xn-1)
        alsmc = pdfmul(Ysys[t+1],multiplication(H,xsmc),V) #calculate incremental weight \alpha_n = g(yn|xn)
        wsmc =  alsmc/np.sum(alsmc) #weight of each particl at time n #wn=\alpha_n * wn-1
        Xsmc[t+1] = xsmc
        Wsmc[t+1] = wsmc
        Xb[t+1] = resam(Xsmc[t+1], Wsmc[t+1], N)
    
    for k in range(0,T):
        Meansmc1[k] = np.sum(Xsmc[k][:,0]*Wsmc[k]) #filtering meanx1 time k
        Meansmc2[k] = np.sum(Xsmc[k][:,1]*Wsmc[k]) #filtering meanx2 time k
    return Xsmc, Wsmc, Meansmc1, Meansmc2

T=100

In [7]:
#plot SIS mean for x1 (position)
Meansis10 = SIS(100,10)[2] #10 particles
Meansis100 = SIS(100,100)[2] #100 particles
Meansis1000 = SIS(100,1000)[2] #1000 particles

fig, ax = plt.subplots(3,1, figsize=(15,15))
fig.suptitle('SIS filtering for x1')

ax[0].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[0].plot(np.arange(start=1, stop=T+1, step=1),Meansis10,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[0].set_title("N=10", fontsize = 12)
ax[0].legend(loc='best',fontsize=15)

ax[1].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[1].plot(np.arange(start=1, stop=T+1, step=1),Meansis100,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[1].set_title("N=100", fontsize = 12)

ax[2].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[2].plot(np.arange(start=1, stop=T+1, step=1),Meansis1000,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[2].set_title("N=1000", fontsize = 12)
ax[2].set_xlabel('time', fontsize=12)

plt.show()

In [8]:
#plot SIS mean for x2 #hidden state(velocity)
Meansis10 = SIS(100,10)[3] #10 particles
Meansis100 = SIS(100,100)[3] #100 particles
Meansis1000 = SIS(100,1000)[3] #1000 particles

fig, ax = plt.subplots(3,1, figsize=(15,15))
fig.suptitle('SIS filtering for x2')

ax[0].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[0].plot(np.arange(start=1, stop=T+1, step=1),Meansis10,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[0].set_title("N=10", fontsize = 12)
ax[0].legend(loc='best',fontsize=15)

ax[1].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[1].plot(np.arange(start=1, stop=T+1, step=1),Meansis100,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[1].set_title("N=100", fontsize = 12)

ax[2].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[2].plot(np.arange(start=1, stop=T+1, step=1),Meansis1000,linestyle='-',linewidth=1,color='red',label='SIS Filter Mean')
ax[2].set_title("N=1000", fontsize = 12)
ax[2].set_xlabel('time', fontsize=12)

plt.show()

In [9]:
#plot SMC mean for x1 (position)
Meansmc10 = SMC(100,10)[2] #10 particles
Meansmc100 = SMC(100,100)[2] #100 particles
Meansmc1000 = SMC(100,1000)[2] #1000 particles

fig, ax = plt.subplots(3,1, figsize=(15,15))
fig.suptitle('SMC filtering for x1')

ax[0].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[0].plot(np.arange(start=1, stop=T+1, step=1),Meansmc10,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[0].set_title("N=10", fontsize = 12)
ax[0].legend(loc='best',fontsize=15)

ax[1].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[1].plot(np.arange(start=1, stop=T+1, step=1),Meansmc100,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[1].set_title("N=100", fontsize = 12)

ax[2].plot(np.arange(start=1, stop=T+1, step=1),mu[0],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[2].plot(np.arange(start=1, stop=T+1, step=1),Meansmc1000,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[2].set_title("N=1000", fontsize = 12)
ax[2].set_xlabel('time', fontsize=12)

plt.show()

In [11]:
#plot SMC mean for x2 #hidden state(velocity)
Meansmc10 = SMC(100,10)[3] #10 particles
Meansmc100 = SMC(100,100)[3] #100 particles
Meansmc1000 = SMC(100,1000)[3] #1000 particles

fig, ax = plt.subplots(3,1, figsize=(15,15))
fig.suptitle('SMC filtering for x2')

ax[0].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[0].plot(np.arange(start=1, stop=T+1, step=1),Meansmc10,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[0].set_title("N=10", fontsize = 12)
ax[0].legend(loc='best',fontsize=15)

ax[1].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[1].plot(np.arange(start=1, stop=T+1, step=1),Meansmc100,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[1].set_title("N=100", fontsize = 12)

ax[2].plot(np.arange(start=1, stop=T+1, step=1),mu[1],marker='+',linestyle='-',color='blue',label='Kalman Filter mean ')
ax[2].plot(np.arange(start=1, stop=T+1, step=1),Meansmc1000,linestyle='-',linewidth=1,color='red',label='SMC Filter Mean')
ax[2].set_title("N=1000", fontsize = 12)
ax[2].set_xlabel('time', fontsize=12)

plt.show()